<a href="https://colab.research.google.com/github/wcarolw/VisaoComputacional/blob/main/Solucao_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow opencv-python

import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def carregar_e_preprocessar(caminho, tamanho=(128, 128)):
    img = cv2.imread(caminho)
    if img is None:
        return None
    img = cv2.resize(img, tamanho)
    img = img.astype("float32") / 255.0
    return img

X = []
y = []

path_healthy = '/content/drive/MyDrive/Visao Computacional/rede neural treinamento/Strawberry___healthy'
path_scorch = '/content/drive/MyDrive/Visao Computacional/rede neural treinamento/Strawberry___Leaf_scorch'

for caminho, label in [(path_healthy, 0), (path_scorch, 1)]:
    for nome_arquivo in os.listdir(caminho):
        img_path = os.path.join(caminho, nome_arquivo)
        img = carregar_e_preprocessar(img_path)
        if img is not None:
            X.append(img)
            y.append(label)

X = np.array(X)
y = np.array(y)


In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)


In [5]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # binário: 0 ou 1
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    X_train, y_train,
    epochs=15,
    batch_size=32,
    validation_split=0.2
)


Epoch 1/15
28/28 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.7092 - loss: 0.6010 - val_accuracy: 0.7568 - val_loss: 0.5140
Epoch 2/15
28/28 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - accuracy: 0.7320 - loss: 0.5245 - val_accuracy: 0.7568 - val_loss: 0.4514
Epoch 3/15
28/28 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - accuracy: 0.7500 - loss: 0.4309 - val_accuracy: 0.8559 - val_loss: 0.2885
Epoch 4/15
28/28 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.8737 - loss: 0.2952 - val_accuracy: 0.8198 - val_loss: 0.3170
Epoch 5/15
28/28 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.8642 - loss: 0.2848 - val_accuracy: 0.9369 - val_loss: 0.1770
Epoch 6/15
28/28 ━━━━━━━━━━━━━━━━━━━━ 44s 1s/step - accuracy: 0.9467 - loss: 0.1696 - val_accuracy: 0.9505 - val_loss: 0.1440
Epoch 7/15
28/28 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step - accuracy: 0.9518 - loss: 0.1560 - val_accuracy: 0.9324 - val_loss: 0.1341
Epoch 8/15
28/28 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.9620 - loss: 0.1233 - val_accuracy: 0.9730 - val_loss:

In [20]:
loss, acc = model.evaluate(X_test, y_test)
print(f"Acurácia no conjunto de teste: {acc:.2f}")

# Predizer uma nova imagem
img_teste = carregar_e_preprocessar('/content/drive/MyDrive/Visao Computacional/teste/6d1f6f9e-facb-4e5e-a0a9-31a30eb8bde4___RS_L.Scorch 0910_final_masked.jpg')
img_teste = np.expand_dims(img_teste, axis=0)

pred = model.predict(img_teste)[0][0]
print("Predição:", "ESCALDADURA" if pred > 0.2 else "SAUDÁVEL")


9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 341ms/step - accuracy: 0.9534 - loss: 0.0981
Acurácia no conjunto de teste: 0.95
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Predição: ESCALDADURA


In [21]:
import matplotlib.pyplot as plt

pasta_teste = '/content/drive/MyDrive/Visao Computacional/rede neural teste'
arquivos = os.listdir(pasta_teste)

for nome in arquivos:
    caminho_img = os.path.join(pasta_teste, nome)
    img = carregar_e_preprocessar(caminho_img)
    if img is not None:
        img_input = np.expand_dims(img, axis=0)
        pred = model.predict(img_input)[0][0]
        classe = "ESCALDADURA" if pred > 0.2 else "SAUDÁVEL"
        print(f"{nome}: {classe} ({pred:.2f})")

        # # opcional: mostrar imagem
        # plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        # plt.title(classe)
        # plt.axis('off')
        # plt.show()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1b9a47df-830d-4c52-b7e0-3180c8fee141___RS_HL 4482.JPG: SAUDÁVEL (0.03)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
08ac4a71-35c5-45c5-8e49-20427b6e53db___RS_HL 4362.JPG: SAUDÁVEL (0.04)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
2e738c4c-8d5c-4c1d-91ad-52dfa3e4ec1f___RS_HL 1768.JPG: SAUDÁVEL (0.05)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1e007f9a-6e90-4dd6-9be9-f0ce3775128f___RS_HL 2076.JPG: SAUDÁVEL (0.04)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1d92fcef-79d0-469b-b9b1-71a318505821___RS_HL 4487.JPG: SAUDÁVEL (0.04)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1a119565-0295-4692-95cd-fbc32728f855___RS_HL 1853.JPG: SAUDÁVEL (0.03)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
06d4ec77-12ef-47ed-ab37-b181fb1b03bf___RS_HL 4647.JPG: SAUDÁVEL (0.03)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
0f79c593-bcf2-4a5b-baac-6433f3037a89___RS_HL 2022.JPG: SAUDÁVEL (0.03)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
2acb16d1-598e-4ee1-9b54-eecfba7d4fc3___RS_HL 1754.JPG: SAUDÁVEL (0.03)
1/1 ━━━━━━━━━━━━━━

In [22]:
import shutil

output_base = '/content/drive/MyDrive/Visao Computacional/classificados'
os.makedirs(os.path.join(output_base, "ESCALDADURA"), exist_ok=True)
os.makedirs(os.path.join(output_base, "SAUDAVEL"), exist_ok=True)

for nome in arquivos:
    caminho_img = os.path.join(pasta_teste, nome)
    img = carregar_e_preprocessar(caminho_img)
    if img is not None:
        img_input = np.expand_dims(img, axis=0)
        pred = model.predict(img_input)[0][0]
        classe = "ESCALDADURA" if pred > 0.2 else "SAUDAVEL"

        destino = os.path.join(output_base, classe, nome)
        shutil.copy(caminho_img, destino)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━